In [1]:
import pickle
import sys
from zoneinfo import ZoneInfo
sys.path.append("../")

from dotenv import load_dotenv
load_dotenv()
import geopandas as gpd
import importlib
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import lightning.pytorch as pl
import rasterio as rio
from rasterio.plot import show
import seaborn as sns
import shapely
import statsmodels.api as sm
from torch.utils.data import DataLoader

from openbustools import plotting, spatial, standardfeeds
from openbustools.traveltime import data_loader, model_utils
from openbustools.drivecycle import trajectory
from openbustools.drivecycle.physics import conditions, energy, vehicle

In [4]:
cleaned_sources = pd.read_csv(Path('..', 'data', 'cleaned_sources.csv'))
cleaned_sources.head()

,country_code,subdivision_name,municipality,provider,static_url,realtime_url,min_lon,max_lon,min_lat,max_lat,uuid,tz_str,epsg_code
0,CA,Ontario,London,London Transit Commission,http://www.londontransit.ca/gtfsfeed/google_tr...,http://gtfs.ltconline.ca/Vehicle/VehiclePositi...,-81.363110,-81.137591,42.905244,43.051188,0fd41f26-6bc5-45f0-88ff-594bde1e8c24,America/Toronto,32617
1,CA,Ontario,Barrie,Barrie Transit,http://www.myridebarrie.ca/gtfs/Google_transit...,http://www.myridebarrie.ca/gtfs/GTFS_VehiclePo...,-79.740632,-79.610896,44.321804,44.420207,3196b95d-1eb2-4e05-bb36-7a2099d2348b,America/Toronto,32617
2,US,California,Santa Monica,Big Blue Bus,http://gtfs.bigbluebus.com/current.zip,http://gtfs.bigbluebus.com/vehiclepositions.bin,-118.549205,-118.237266,33.929498,34.075669,dbeb97e3-012d-4c23-93aa-a80696884340,America/Los_Angeles,32611
3,US,California,Mountain View,Mountain View Transportation Management Associ...,http://data.trilliumtransit.com/gtfs/mountainv...,https://ridemvgo.org/gtfs-rt/vehiclepositions,-122.111591,-122.047584,37.387656,37.431429,6885d33d-2f4e-4ac7-84a7-2e4747c723f0,America/Los_Angeles,32610
4,US,California,Merced,Merced County Transit (The Bus),http://data.trilliumtransit.com/gtfs/mercedthe...,https://thebuslive.com/gtfs-rt/vehiclepositions,-121.020240,-120.248000,36.964503,37.521786,ea803659-c28f-43ac-8c01-619d767e0b1d,America/Los_Angeles,32610


In [46]:
all_data = []
for uuid in cleaned_sources.uuid:
    provider_path = Path('..', 'data', 'other_feeds', f"{uuid}_realtime")
    available_files = [x.name for x in provider_path.glob('*.pkl')]
    for fname in available_files:
        data = pd.read_pickle(provider_path / fname)
        data['uuid'] = uuid
        all_data.append(data)
all_data = pd.concat(all_data)
all_data = pd.merge(cleaned_sources[['uuid', 'municipality', 'provider']], all_data, on='uuid')
all_data.head()

,uuid,municipality,provider,trip_id,file,locationtime,lat,lon,vehicle_id
0,0fd41f26-6bc5-45f0-88ff-594bde1e8c24,London,London Transit Commission,2038880,2023_12_31,1704038914,42.963730,-81.231598,3507
1,0fd41f26-6bc5-45f0-88ff-594bde1e8c24,London,London Transit Commission,2038880,2023_12_31,1704038944,42.961960,-81.231133,3507
2,0fd41f26-6bc5-45f0-88ff-594bde1e8c24,London,London Transit Commission,2038880,2023_12_31,1704038974,42.961739,-81.230949,3507
3,0fd41f26-6bc5-45f0-88ff-594bde1e8c24,London,London Transit Commission,2038880,2023_12_31,1704039004,42.961788,-81.229881,3507
4,0fd41f26-6bc5-45f0-88ff-594bde1e8c24,London,London Transit Commission,2038880,2023_12_31,1704039036,42.962170,-81.225677,3507


In [47]:
all_data.groupby('provider').count().sort_values('uuid', ascending=False)

,uuid,municipality,trip_id,file,locationtime,lat,lon,vehicle_id
provider,,,,,,,,
King County Metro,483728,483728,483728,483728,483728,483728,483728,483728
Massachusetts Bay Transportation Authority (MBTA),386814,386814,386814,386814,386814,386814,386814,386814
Edmonton Transit System,347690,347690,347690,347690,347690,347690,347690,347690
Capital Metro,300917,300917,300917,300917,300917,300917,300917,300917
Port Authority of Allegheny County,278327,278327,278327,278327,278327,278327,278327,278327
Metropolitan Atlanta Rapid Transit Authority (MARTA),177917,177917,177917,177917,177917,177917,177917,177917
MiWay,148117,148117,148117,148117,148117,148117,148117,148117
Central Florida Regional Transit Authority (LYNX),143568,143568,143568,143568,143568,143568,143568,143568
Hamilton Street Railway,115748,115748,115748,115748,115748,115748,115748,115748
